In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Análise dados em base de clientes.

#### Introdução.
     Notebook com base de dados de crédito de clientes. Vamos tratar, explorar e analisar as informações contidas nos dados.

#### 1. Exploração dos dados

Nessa primeira etapa vamos pegar nossos dados, analisar algumas informações iniciais como os tipos da varáveis, dados faltantes e decrições.

In [ ]:
#Importanto biblioteca Pandas e lendo nossa base de dados
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/develop/dataset/credito.csv', na_values = 'na')
df.head()

In [ ]:
df.info()

* Analisando variáveis categóricas.

In [ ]:
df.select_dtypes('object').describe().transpose()

* Analisando variáveis numéricas.

In [ ]:
df.select_dtypes('number').describe().transpose()

* removendo a coluna ID para melhor visualização.

In [ ]:
df.drop('id', axis = 1).select_dtypes('number').describe().transpose()

As colunas "limite_credito" e "valor_transacoes_12m" estão como string, vamos passar para float.

In [ ]:
df['limite_credito'] = df['limite_credito'].apply(lambda valor: float(valor.replace('.', '').replace(',', '.')))
df['valor_transacoes_12m'] = df['valor_transacoes_12m'].apply(lambda valor: float(valor.replace('.', '').replace(',', '.')))

In [ ]:
df.drop('id', axis = 1).select_dtypes('number').describe().transpose()

In [ ]:
df.head()

* Descobrindo a proporção de clientes adimplentes e inadimplentes.

Estamos guardando nas variáveis abaixo, o total de clientes, clientes adimplentes e clientes inadimplentes:

In [ ]:
qtd_total, _ = df.shape
qtd_adimplentes, _ = df[df['default'] == 0].shape
qtd_inadimplentes, _ = df[df['default'] == 1].shape

In [ ]:
print(f'Proporção de clientes adimplentes: {round(100 * qtd_adimplentes / qtd_total, 2)}%')
print(f'Proporção de clientes inadimplentes: {round(100 * qtd_inadimplentes / qtd_total, 2)}%')

#### 2. Tratamento dos dados.
* Removendo valores nulos

In [ ]:
df.isna().any()

Como podemos ver, temos 3 colunas com dados faltantes: "escolaridade", "estado_civil" e "salario_anual".

In [ ]:
print(df['escolaridade'].shape)
print(df['estado_civil'].shape)
print(df['salario_anual'].shape)

Função para filtrar dados faltantes:

In [ ]:
def status_dados_faltantes(df):
    dados_faltantes = []
    
    for col in df.columns:
        if df[col].isna().any():
            qtd, _ = df[df[col].isna()].shape
            total, _ = df.shape
            dic_dados_faltantes = {col: {'Quantidade:': qtd, 'Porcentagem:': round(100 * qtd / total, 2)}}
            dados_faltantes.append(dic_dados_faltantes)
            
    for status in dados_faltantes:
        print(status)

In [ ]:
status_dados_faltantes(df)

In [ ]:
df.shape

Removendo valores:

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.shape

Atualizando as variáveis sem os dados faltantes:

In [ ]:
qtd_total_novo, _ = df.shape
qtd_adimplentes_novo, _ = df[df['default'] == 0].shape
qtd_inadimplentes_novo, _ = df[df['default'] == 1].shape

In [ ]:
print(f'Quantidade de adimplentes: {round(100 * qtd_adimplentes / qtd_total ,2)}%')
print(f'Quantidade de adimplentes atualizada: {round(100 * qtd_adimplentes_novo / qtd_total_novo ,2)}%')
print('')
print(f'Quantidade de inadimplentes: {round(100 * qtd_inadimplentes / qtd_total ,2)}%')
print(f'Quantidade de inadimplentes atualizada: {round(100 * qtd_inadimplentes_novo / qtd_total_novo ,2)}%')

### 3. Visualização dos dados.

3.1 - Visualizações categóricas.

In [ ]:
# Importando bibiotlecas de visualização.
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')

In [ ]:
df.select_dtypes('object').head()

Filtrando clientes adimplentes e inadimplentes:

In [ ]:
df_adimplente = df[df['default'] == 0]

In [ ]:
df_inadimplente = df[df['default'] == 1]

In [ ]:
display(df_adimplente.head())
display(df_inadimplente.head())

* Escolaridade.

In [ ]:
coluna = 'escolaridade'
titulos = ['Escolaridade dos Clientes', 'Escolaridade dos Clientes Adimplentes', 'Escolaridade dos Clientes Inadimplentes']

eixo = 0
max_y = 0
max = df.select_dtypes('object').describe()[coluna]['freq'] * 1.1

figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.sort_values(by=[coluna], inplace=True)
  df_to_plot.sort_values(by=[coluna])

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

* Salário anual.

In [ ]:
coluna = 'salario_anual'
titulos = ['Salário Anual dos Clientes', 'Salário Anual dos Clientes Adimplentes', 'Salário Anual dos Clientes Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  df_to_plot = dataframe[coluna].value_counts().to_frame()
  df_to_plot.rename(columns={coluna: 'frequencia_absoluta'}, inplace=True)
  df_to_plot[coluna] = df_to_plot.index
  df_to_plot.reset_index(inplace=True, drop=True)
  df_to_plot.sort_values(by=[coluna], inplace=True)

  f = sns.barplot(x=df_to_plot[coluna], y=df_to_plot['frequencia_absoluta'], ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')
  f.set_xticklabels(labels=f.get_xticklabels(), rotation=90)
  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))
  eixo += 1

figura.show()

3.2 - Visualizações numéricas.

In [ ]:
df.drop(['id', 'default'], axis = 1).select_dtypes('number').head()

* Quantidade de transações nos últimos 12 meses.

In [ ]:
coluna = 'qtd_transacoes_12m'
titulos = ['Qtd. de Transações no Último Ano', 'Qtd. de Transações no Último Ano de Adimplentes', 'Qtd. de Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

* Valor das transações nos últimos 12 meses.

In [ ]:
coluna = 'valor_transacoes_12m'
titulos = ['Valor das Transações no Último Ano', 'Valor das Transações no Último Ano de Adimplentes', 'Valor das Transações no Último Ano de Inadimplentes']

eixo = 0
max_y = 0
figura, eixos = plt.subplots(1,3, figsize=(20, 5), sharex=True)

for dataframe in [df, df_adimplente, df_inadimplente]:

  f = sns.histplot(x=coluna, data=dataframe, stat='count', ax=eixos[eixo])
  f.set(title=titulos[eixo], xlabel=coluna.capitalize(), ylabel='Frequência Absoluta')

  _, max_y_f = f.get_ylim()
  max_y = max_y_f if max_y_f > max_y else max_y
  f.set(ylim=(0, max_y))

  eixo += 1

figura.show()

* Transações nos Últimos 12 Meses x Quantidade de Transações nos Últimos 12 Meses.


In [ ]:
f = sns.relplot(x = 'valor_transacoes_12m', y = 'qtd_transacoes_12m', data = df, hue = 'default')
_ = f.set(title = 'Relação entre Valor e Quantidade de Transações no Último Ano',
                    xlabel = 'Valor das Transações no Último Ano',
                   ylabel = 'Quantidade das Transações no Último Ano')

## Conlcusão:
Como podemos observar nos gráficos, nosso quadro de inadimplentes está na faixa de menos de 40k por ano e quadro de escolaridade no mestrado.